In [ ]:
# | default_exp routes.publish

In [ ]:
# | exporti
import httpx

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda

# Publications

In [ ]:
# | export
async def search_publications(auth: dmda.DomoAuth,
                              search_term: str = None, 
                              limit=100, offset=0,
                              session: httpx.AsyncClient = None, debug_api: bool = False) -> rgd.ResponseGetData:
    url = f"https://{auth.domo_instance}.domo.com/api/publish/v2/publication/summaries"

    params = {'limit': limit, 'offset': offset}

    if search_term:
        params.update({'searchTerm': search_term})

    res = await gd.get_data(auth=auth,
                         method='GET',
                         url=url,
                         params=params,
                         session=session,
                         debug_api=debug_api)

    return res


In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community", 
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

res = await search_publications(auth=token_auth)

pd.DataFrame(res.response[0:5])


,id,name,description,dataSets,pages,cards,subscribers,totalSubscribers,invalidSubscribers,invalidDatasetSubscribers,invalidPageSubscribers,invalidCardSubscribers,isV2,hasDuplicate,refreshing,executionType,executionEstimatedCompletionTime,executionStartTime,executionEndTime
0,6ea21fc2-ca94-4235-9576-598421a0f36b,Test OZ,,1,0,0,0,1,0,0,0,0,True,False,False,NEW_PUBLICATION,1683558947835,1683558857839,1683558858376
1,c74e1958-e436-4f71-930f-21af39b50cb6,Test OZ,,1,0,0,0,1,0,0,0,0,True,False,False,NEW_PUBLICATION,1684942911622,1684942891622,1684942892118
2,cd011de1-a88a-429c-be72-9011f93e9f1e,Test OZ,,1,0,0,0,1,0,0,0,0,True,False,False,NEW_PUBLICATION,1685026095900,1685026075900,1685026076606
3,bd4d5598-6586-4301-b8f3-0a0dbf8a20e6,Test OZ,,1,0,0,0,1,0,0,0,0,True,False,False,NEW_PUBLICATION,1684943341187,1684943321187,1684943321703
4,093089ef-d0e3-45f4-8d6e-0208a7108a51,Test OZ,,1,0,0,0,1,0,0,0,0,True,False,False,NEW_PUBLICATION,1684944342458,1684944322458,1684944322994


In [ ]:
# | export
async def get_publication_by_id(auth: dmda.DomoAuth,
                                publication_id: str,
                                session: httpx.AsyncClient = None, debug_api: bool = False,
                                timeout = 10
                                ) -> rgd.ResponseGetData:
    url = f"https://{auth.domo_instance}.domo.com/api/publish/v2/publication/{publication_id}"

    res = await gd.get_data(auth=auth,
                            method='GET',
                            url=url,
                            session=session,
                            timeout = 10,
                            debug_api=debug_api)

    return res


# generate publish body


In [ ]:
import os
import pandas as pd

publication_id = "701eb547-4244-43f6-bcda-003ed2c483c7"

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

res = await get_publication_by_id(publication_id = publication_id,
                                  auth=token_auth, timeout = 10)

res.response


{'id': '701eb547-4244-43f6-bcda-003ed2c483c7',
 'name': 'test_publish',
 'customerId': 'mmmm-0012-0200',
 'created': 1682580980642,
 'content': {'id': 'dc7b65dd-28fc-4fa6-b469-26873c668549',
  'domain': 'domo-community.domo.com',
  'customerId': 'mmmm-0012-0200',
  'userId': '612085674',
  'type': 'PACKAGE',
  'updated': 1682580980642,
  'hash': '7b93d001-ba90-4975-b79d-690289673ae1',
  'publicationId': '701eb547-4244-43f6-bcda-003ed2c483c7'},
 'description': '',
 'isV2': True,
 'hasDuplicate': False,
 'useDirectContent': True,
 'type': 'CONTENT',
 'isBackup': False,
 'children': [{'id': 'f80ef8c8-a133-439c-b459-7967ce7df4b3',
   'customerId': '',
   'created': 1685653334877,
   'content': {'id': '7234b90b-09d9-48cb-876a-d8ed11fc6c2a',
    'domain': 'domo-community.domo.com',
    'customerId': '',
    'userId': '612085674',
    'domoObjectId': 'e17fd398-8abf-4cb5-8ed1-5394e274fbf4',
    'type': 'DATASET',
    'updated': 1685653334877,
    'hash': 'c33fb089-a0eb-419e-b449-67654178fd3d'}

In [ ]:
#| export
def generate_publish_body(url: str, sub_domain_ls: [str], content_ls: [str], name: str, description: str, unique_id: str, is_new: bool):
    if not sub_domain_ls:
        sub_domain_ls = []

    if not content_ls:
        content_ls = []

    body = {
        "id": unique_id,
        "name": name,
        "description": description,
        "domain": url,
        "content": content_ls,
        "subscriberDomain": sub_domain_ls,
        "new": str(is_new).lower()
    }

    return body

In [ ]:
import os
import pandas as pd

publication_id = "701eb547-4244-43f6-bcda-003ed2c483c7"

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)
body = generate_publish_body("domo-community.domo.com",["test.domo.com"],["content"],"test",'','id',True)
body

{'id': 'id',
 'name': 'test',
 'description': '',
 'domain': 'domo-community.domo.com',
 'content': ['content'],
 'subscriberDomain': ['test.domo.com'],
 'new': 'true'}

In [ ]:
#| export
# Creating publish job for a specific subscriber
async def create_publish_job(auth: dmda.DomoAuth, body : dict, session: httpx.AsyncClient = None, debug_api: bool = False) -> rgd.ResponseGetData:
    url = f'https://{auth.domo_instance}.domo.com/api/publish/v2/publication'

    if (debug_api):
        print (body)
    res = await gd.get_data(auth=auth,
                         method='POST',
                         url=url,
                         body=body,
                         session=session,
                         debug_api=debug_api)

    return res




In [ ]:
#| export
# Updating existing publish job with content
async def udpate_publish_job(auth: dmda.DomoAuth, publication_id: str, body: dict, session: httpx.AsyncClient = None, debug_api: bool = False) -> rgd.ResponseGetData:

    url = f'https://{auth.domo_instance}.domo.com/api/publish/v2/publication/{publication_id}'

    res = await gd.get_data(auth=auth,
                         method='PUT',
                         url=url,
                         body=body,
                         session=session,
                         debug_api=debug_api)
    return res

# # finds all jobs waiting to be accepted within the subscriber

# Subscriptions


In [ ]:
# | export
async def get_subscription_summaries(
    auth: dmda.DomoAuth, session: httpx.AsyncClient = None, debug_api: bool = False
) -> rgd.ResponseGetData:

    """retrieves a summary of existing subscriptions"""
    url = f"https://{auth.domo_instance}.domo.com/api/publish/v2/subscription/summaries"

    res = await gd.get_data(
        auth=auth, method="GET", url=url, session=session, debug_api=debug_api
    )
    return res

In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

res = await get_subscription_summaries(auth=token_auth)

pd.DataFrame(res.response[0:5])

""


In [ ]:
# | export
async def get_subscription_invititations(
    auth: dmda.DomoAuth, session: httpx.AsyncClient = None, debug_api: bool = False
) -> rgd.ResponseGetData:
    """retrieves a list of subscription invitations"""

    url = f"https://{auth.domo_instance}.domo.com/api/publish/v2/subscription/invites"

    res = await gd.get_data(
        auth=auth, method="GET", url=url, session=session, debug_api=debug_api
    )
    return res

#### sample implementation of get_subscription_invitations

In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

res = await get_subscription_invititations(auth=token_auth)

pd.DataFrame(res.response[0:5])

""


In [ ]:
# | export
async def accept_invite_by_id(auth: dmda.DomoAuth,
                              subscription_id: str, 
                              session: httpx.AsyncClient = None, debug_api: bool = False) -> rgd.ResponseGetData:
    """this takes get_subscription_invites_list into account and accepts - not instant"""

    url = f'https://{auth.domo_instance}.domo.com/api/publish/v2/subscription/{subscription_id}'

    res = await gd.get_data(auth=auth,
                        method='POST',
                        url=url,
                        session=session,
                        debug_api=debug_api)
    return res

In [ ]:
# | export
async def refresh_publish_jobs(auth: dmda.DomoAuth,
                               publish_ids: list,
                               session: httpx.AsyncClient = None, debug_api: bool = False) -> rgd.ResponseGetData:
    """Refreshing list of publish jobs. Typically "instance" = publisher instance"""

    url = f'https://{auth.domo_instance}.domo.com/api/publish/v2/publication/refresh'

    body = {
        'publicationIds': publish_ids
    }

    res = await gd.get_data(auth=auth,
                         method='PUT',
                         url=url,
                         body=body,
                         session=session,
                         debug_api=debug_api)
    return res


In [ ]:
#| hide

import nbdev
nbdev.nbdev_export()